In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ILUMINA_PATH = Path('../data/Ilumina')
ILUMINA_CLEAN_PATH = Path('../data/Ilumina/clean')

NANOPORE_PATH = Path('../data/Nanopore')
NANOPORE_CLEAN_PATH = Path('../data/Nanopore/clean')

In [3]:
IluminaPylum = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Pylum-level microbiota")
IluminaFamily = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Family-level microbiota")
IluminaGenus = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Genus-level microbiota")

NanoporePylum = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Pylum-level microbiota")
NanoporeFamily = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Family-level microbiota")
NanoporeGenus = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Genus-level microbiota")

In [4]:
IluminaPylum["Fertil"] = 1
IluminaPylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaFamily["Fertil"] = 1
IluminaFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaGenus["Fertil"] = 1
IluminaGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporePylum["Fertil"] = 1
NanoporePylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeFamily["Fertil"] = 1
NanoporeFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeGenus["Fertil"] = 1
NanoporeGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

In [6]:
IluminaPylum.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylum.csv", index=False, encoding='utf-8')
IluminaFamily.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamily.csv", index=False, encoding='utf-8')
IluminaGenus.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenus.csv", index=False, encoding='utf-8')

NanoporePylum.to_csv(NANOPORE_CLEAN_PATH / "NanoporePylum.csv", index=False, encoding='utf-8')
NanoporeFamily.to_csv(NANOPORE_CLEAN_PATH / "NanoporeFamily.csv", index=False, encoding='utf-8')
NanoporeGenus.to_csv(NANOPORE_CLEAN_PATH / "NanoporeGenus.csv", index=False, encoding='utf-8')